In [1]:
import os
from brtdevkit.util.aws import s3

Internal AWS credentials will be removed from future brt-devkit versions.


In [2]:
s3_client = s3.S3()
BRT_DEVKIT_WRITE_S3_BUCKET = 'mesa-states'
MODEL_TRAINING_S3_KEY_PREFIX = f"{os.environ.get('BRT_ENV', 'prod')}/jupiter/model_training"
PACK_PERCEPTION_S3_KEY_PREFIX = f"{os.environ.get('BRT_ENV', 'prod')}/jupiter/pack_perception"
bucket_name=BRT_DEVKIT_WRITE_S3_BUCKET

In [ ]:
# upload file
# key = f"{MODEL_TRAINING_S3_KEY_PREFIX}/Jupiter_train_v4_53_missing_human_relabeled.tar"
key = f"{PACK_PERCEPTION_S3_KEY_PREFIX}/Jupiter_train_v4_53_heavy_dust_relabeled.tar"
file_path = "/home/bluerivertech/li.yu/data/Jupiter_train_v4_53_heavy_dust_relabeled/Jupiter_train_v4_53_heavy_dust_relabeled.tar"
s3_client.upload_file(
    bucket_name=bucket_name, key=key, file_path=file_path)
# Jupiter_2022_fn_bag1.tar
# Jupiter_2022Jan15Feb15_rock_stops.tar
# Jupiter_train_v4_53_missing_human_relabeled.tar
# Jupiter_train_v4_53_heavy_dust_relabeled.tar

In [4]:
# download file
# run_id = '45_3_6class_ml_pp_seg_lite_cloud_v10_rotate_fix'
# name = 'driveable_terrain_model_val_bestmodel.pth'
# key = f"{MODEL_TRAINING_S3_KEY_PREFIX}/{run_id}/{name}"

key = f"{PACK_PERCEPTION_S3_KEY_PREFIX}/Jupiter_train_v4_53_heavy_dust_relabeled.tar"

target_path = '/data/jupiter/li.yu/data/Jupiter_train_v4_53/Jupiter_train_v4_53_heavy_dust_relabeled/Jupiter_train_v4_53_heavy_dust_relabeled.tar'
s3_client.download_file(bucket_name, key, target_path)

True